Peter Ling
NAÏVE BAYES ALGORITHM

In [ ]:
import csv
import numpy as np
import pandas as pd
import math
from operator import itemgetter

In my implementation of the naive bayes algorithm, I take in training data as a dataframe in which the parameters are the columns and each row is a different data point. The last column will be the output, which will be 0 or 1.

Naïve Bayes can be done with either MLE or Laplace estimation, and my function allows the user to choose their preference with the laplace parameter to the get_thetas function. laplace = True would use laplace estimation, while laplace = False would train the model using MLE.

In [ ]:
def get_conditional_probabilities(training_df, laplace): 
    
    n_params = len(training_df.columns) - 1
    n_examples = len(training_df)

    c_probs = []

    for param in range(n_params): # 0 then 1 then 2... 0indexed

        param0_given_0 = 0
        param1_given_0 = 0

        param0_given_1 = 0
        param1_given_1 = 0

        count_y0 = 0
        count_y1 = 0

        row = 0
        while row < n_examples:

            training_df_value = training_df.iloc[row][param]
            output = training_df.iloc[row][n_params]
            if (training_df.iloc[row][n_params] == 0): # CHECK IF OUTPUT IS 0
                count_y0 += 1
                if training_df.iloc[row][param] == 0:
                    param0_given_0 += 1
                else:
                    param1_given_0 += 1

            else: # OUTPUT IS 1
                count_y1 += 1
                if training_df.iloc[row][param] == 0:
                    param0_given_1 += 1
                else:
                    param1_given_1 += 1

            row += 1
            
            
        if laplace:
            c_probs.append([(param0_given_0 + 1) / (count_y0 + 2), (param0_given_1 + 1) / (count_y1 + 2), (param1_given_0 + 1) / (count_y0 + 2), (param1_given_1 + 1) / (count_y1 + 2)])
        else:
            c_probs.append([param0_given_0 / count_y0, param0_given_1 / count_y1, param1_given_0 / count_y0, param1_given_1 / count_y1])
        count_training_examples = count_y0 + count_y1
        
        
    return c_probs


After get_conditional_probabilities() has run, it will return a 2d list. Within the large list, each list will represent 

In [ ]:
def test_naive_bayes(test_df, c_probs, laplace): 
    n_params = len(test_df.columns) - 1
    n_tests = len(test_df)
    predictions = []
    
    row = 0
    while row < n_tests: # ROW = EXAMPLE
        log_sum_y0 = 0
        log_sum_y1 = 0

        product_y0 = 1
        product_y1 = 1
        for param in range(n_params):
            param_probs = c_probs[param] # Gives the probabilities for the given parameter


            # First we need to see what the value of the test's parameter is
            location_value = test_df.iloc[row][param]
            if test_df.iloc[row][param] == 0:
                if param_probs[0] != 0:
                    log_sum_y0 += math.log(param_probs[0])

                elif not laplace:
                    log_sum_y0 += math.log(0.001)


                if param_probs[1] != 0:
                    log_sum_y1 += math.log(param_probs[1])

                elif not laplace:
                    log_sum_y1 += math.log(0.001)

            else:
                if param_probs[2] != 0:
                    log_sum_y0 += math.log(param_probs[2])

                elif not laplace:
                    log_sum_y0 += math.log(0.001)

                if param_probs[3] != 0:
                    log_sum_y1 += math.log(param_probs[3])

                elif not laplace:
                    log_sum_y1 += math.log(0.001)


        prob_y0 = math.log(count_y0 / count_training_examples) + log_sum_y0
        prob_y1 = math.log(count_y1 / count_training_examples) + log_sum_y1
        
        
        if (prob_y0 > prob_y1):
            predictions.append(0)
        else:
            predictions.append(1)
            
        row += 1
        